In [1]:
import tensorflow as tf
import numpy as np
import efficientnet.keras as efn 
from efficientnet.keras import preprocess_input

from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, Callback
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input
from keras import backend as K

import random
import cv2 as cv
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
batch_size_all    = 5

In [4]:
base_model = efn.EfficientNetB3(weights=None)
base_model.layers.pop()
base_model.layers.pop()

x = Dense(4, activation='sigmoid')(base_model.layers[-1].output)

model = Model(inputs=base_model.input, outputs=[x])

#for the second and third round load model with the best weights
#model.load_weights('D:/data-petr/signate_3rd_ai_edge/refiner/refiner_ep016-loss1.506-val_loss1.223.h5')
model.load_weights(r'D:\SIGNATE\Signate_3rd_AI_edge_competition\refiner_model/refiner_ep003-loss_2.959-val_loss_2.769.h5')

In [5]:
def box_iou(b1, b2):
    #format: batch, x1, y1, x2, y2
    b1_mins  = b1[..., 0:2]
    b1_maxes = b1[..., 2:4]
    b1_wh    = b1_maxes - b1_mins

    b2_mins  = b2[..., 0:2]
    b2_maxes = b2[..., 2:4]
    b2_wh    = b2_maxes - b2_mins


    intersect_mins  = K.maximum(b1_mins, b2_mins)
    intersect_maxes = K.minimum(b1_maxes, b2_maxes)
    intersect_wh    = K.maximum(intersect_maxes - intersect_mins, 0.)
    intersect_area  = intersect_wh[..., 0] * intersect_wh[..., 1]
    b1_area = b1_wh[..., 0] * b1_wh[..., 1]
    b2_area = b2_wh[..., 0] * b2_wh[..., 1]
    iou = intersect_area / (b1_area + b2_area - intersect_area)
    iou = iou+0.00001 #iou is [0,1], so the constant helps to avoid inf

    return K.sum(-K.log(iou)) 


def box_iou_np(b1, b2):
    
    b1_xy = b1[..., :2]
    b1_wh = b1[..., 2:4] - b1_xy
    b1_mins = b1_xy
    b1_maxes = b1_xy + b1_wh

    b2_xy = b2[..., :2]
    b2_wh = b2[..., 2:4] - b2_xy
    b2_mins = b2_xy
    b2_maxes = b2_xy + b2_wh

    intersect_mins = np.maximum(b1_mins, b2_mins)
    intersect_maxes = np.minimum(b1_maxes, b2_maxes)
    intersect_wh = np.maximum(intersect_maxes - intersect_mins, 0.)
    intersect_area = intersect_wh[..., 0] * intersect_wh[..., 1]
    b1_area = b1_wh[..., 0] * b1_wh[..., 1]
    b2_area = b2_wh[..., 0] * b2_wh[..., 1]
    iou = intersect_area / (b1_area + b2_area - intersect_area)

    return 1.0 - (np.sum(iou) / b1.shape[0])

In [6]:
with open(r'D:\SIGNATE\Signate_3rd_AI_edge_competition\data_for_refiner_training.txt') as f:
    lines = f.readlines()

with open(r'D:\SIGNATE\Signate_3rd_AI_edge_competition\data_for_refiner_validation.txt') as f:
    lines_val = f.readlines()

num_val = int(len(lines_val))
num_train = len(lines)
print('train on', num_train, 'samples;  valid on', num_val, 'samples')





def get_random_data(annotation_line, apply_random=True):
    line = annotation_line.split()

    image = cv.cvtColor(cv.imread(line[0]), cv.COLOR_BGR2RGB)
    box = np.array([np.array(list(map(float, box.split(',')))) for box in line[1:]])

    if apply_random == False:  
        return image, [box[0][0], box[0][1], box[0][2], box[0][3]]

    hue=20
    sat=40 
    val=50
    
    if random.random() < .2:
        clahe = cv.createCLAHE(clipLimit=2, tileGridSize=(8,8))
        lab = cv.cvtColor(image, cv.COLOR_RGB2LAB)
        l, a, b = cv.split(lab)
        cl = clahe.apply(l)
        limg = cv.merge((cl,a,b))
        image = cv.cvtColor(limg, cv.COLOR_LAB2RGB)
        
    #shift   
    new_image = np.full((300,300,3), 0, dtype='uint8')
    sx = random.randint(-30, 40)
    sy = random.randint(-30, 40)
    sx_float = sx/300.0
    sy_float = sy/300.0
    new_image[max(0, sy):min(300, 300+sy), max(0, sx):min(300, 300+sx), ...] = image[max(0, -sy):min(300, 300-sy), max(0,-sx):min(300, 300-sx), ...]
    image = new_image
    box[0][0] = max(min(box[0][0] + sx_float, 1.0), 0.0)
    box[0][1] = max(min(box[0][1] + sy_float, 1.0), 0.0)
    box[0][2] = max(min(box[0][2] + sx_float, 1.0), 0.0)
    box[0][3] = max(min(box[0][3] + sy_float, 1.0), 0.0)


    #flip image or not
    flip = random.random() < .5
    if flip:  image = cv.flip(image, 1)

    # distort image
    hsv = np.int32(cv.cvtColor(image, cv.COLOR_RGB2HSV))
    
    #linear hsv distortion
    hsv[..., 0] += random.randint(-hue, hue)
    hsv[..., 1] += random.randint(-sat, sat)
    hsv[..., 2] += random.randint(-val, val)
    
    #additional non-linear distortion of saturation and value
    if random.random()<0.4:
        hsv[..., 1] = hsv[..., 1]*random.uniform(.5, 1.5)
        hsv[..., 2] = hsv[..., 2]*random.uniform(.5, 1.5)
        
    hsv[..., 0][hsv[..., 0] > 179] = 179
    hsv[..., 0][hsv[..., 0] < 0]   = 0
    hsv[..., 1][hsv[..., 1] > 255] = 255
    hsv[..., 1][hsv[..., 1] < 0]   = 0
    hsv[..., 2][hsv[..., 2] > 255] = 255
    hsv[..., 2][hsv[..., 2] < 0]   = 0
    
    image = cv.cvtColor(np.uint8(hsv), cv.COLOR_HSV2RGB)


    box_data = np.zeros((1, 4))
    if len(box) > 0:
        box[:, [0, 2]] = box[:, [0, 2]]
        box[:, [1, 3]] = box[:, [1, 3]]
        if flip: box[:, [0, 2]] = 1.0 - box[:, [2, 0]]
            
    return image, [box[0][0], box[0][1], box[0][2], box[0][3]]






def data_generator(annotation_lines, batch_size, is_random):
    """data generator for fit_generator"""
    n = len(annotation_lines)
    i = 0
    while True:
        image_data = []
        box_data = []
        for b in range(batch_size):
            if i == 0: np.random.shuffle(annotation_lines)
            image, box = get_random_data(annotation_lines[i], apply_random=is_random)
            image_data.append(image)
            box_data.append(box)
            i = (i + 1) % n
        image_data = np.array(image_data)
        y_true = np.array(box_data)
        yield image_data, y_true


def data_generator_wrapper(annotation_lines, batch_size, random):
    n = len(annotation_lines)
    if n == 0 or batch_size <= 0: return None
    return data_generator(annotation_lines, batch_size, random)

train on 2303818 samples;  valid on 29425 samples


In [7]:

model.compile(optimizer=Adadelta(1.0), loss=box_iou)


checkpoint      = ModelCheckpoint(r'D:\SIGNATE\Signate_3rd_AI_edge_competition\refiner_model/refiner_ep{epoch:03d}-loss_{loss:.3f}-val_loss_{val_loss:.3f}.h5', monitor='val_loss', save_weights_only=True, save_best_only=False,verbose=1)
reduce_lr       = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, delta=0.03)


model.fit_generator(data_generator_wrapper(lines, batch_size_all, True),
                      steps_per_epoch=1000,
                      validation_data=data_generator_wrapper(lines_val, batch_size_all, False),
                      validation_steps=max(1, num_val // batch_size_all),
                      epochs=50,
                      callbacks=[reduce_lr, checkpoint])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/50
1000/1000 [==============================] - 912s 912ms/step - loss: 2.7339 - val_loss: 2.0510

Epoch 00001: saving model to D:\SIGNATE\Signate_3rd_AI_edge_competition\refiner_model/refiner_ep001-loss_2.734-val_loss_2.051.h5
Epoch 2/50
1000/1000 [==============================] - 886s 886ms/step - loss: 2.5561 - val_loss: 1.8424

Epoch 00002: saving model to D:\SIGNATE\Signate_3rd_AI_edge_competition\refiner_model/refiner_ep002-loss_2.556-val_loss_1.842.h5
Epoch 3/50
1000/1000 [==============================] - 892s 892ms/step - loss: 2.4066 - val_loss: 2.0305

Epoch 00003: saving model to D:\SIGNATE\Signate_3rd_AI_edge_competition\refiner_model/refiner_ep003-loss_2.407-val_loss_2.031.h5
Epoch 4/50
1000/1000 [==============================] - 893s 893ms/step - loss: 2.3238 - val_loss: 2.4732

Epoch 00004: saving model to D:\SIGNATE\Signate_3rd_AI_edge_competition\refiner_model/r

KeyboardInterrupt: 